In [4]:
%reload_ext autoreload
%autoreload 2

import xarray as xr
from lkf_tools.dataset import *
import dill
import warnings
warnings.filterwarnings('ignore')

In [8]:
lkf_data = process_dataset(f'data/ice_fesom_1950.nc', output_path=f'./data/lkfs/',
                          dog_thres=0.05, t_red=1)

  -->  Compute dxu and dyu from lon,lat using SSMI projection


In [ ]:
%%time
lkf_data.detect_lkfs(indexes=np.arange(365))

Compute deformation rates and detect features for day 1
Start detection routines
Compute deformation rates and detect features for day 2
Start detection routines
Compute deformation rates and detect features for day 3
Start detection routines
Compute deformation rates and detect features for day 4
Start detection routines
Compute deformation rates and detect features for day 5
Start detection routines
Compute deformation rates and detect features for day 6
Start detection routines
Compute deformation rates and detect features for day 7
Start detection routines
Compute deformation rates and detect features for day 8
Start detection routines
Compute deformation rates and detect features for day 9
Start detection routines
Compute deformation rates and detect features for day 10
Start detection routines
Compute deformation rates and detect features for day 11
Start detection routines
Compute deformation rates and detect features for day 12
Start detection routines
Compute deformation rates

In [55]:
for i in range(len(lkf_data.indexes)):
    import matplotlib.pyplot as plt
    import cartopy.crs as ccrs

    fig = plt.figure(figsize=[16, 8])

    ax = plt.subplot(1, 1, 1, projection=ccrs.Orthographic(0, 90))

    ax.coastlines(zorder=3)

    it = lkf_data.indexes[i]

    pcm = ax.pcolormesh(lkf_data.lon[max([0,lkf_data.index_y[0][0]-1]):lkf_data.index_y[0][-1]+2:lkf_data.red_fac,
                           max([0,lkf_data.index_x[0][0]-1]):lkf_data.index_x[0][-1]+2:lkf_data.red_fac],
                  lkf_data.lat[max([0,lkf_data.index_y[0][0]-1]):lkf_data.index_y[0][-1]+2:lkf_data.red_fac,
                           max([0,lkf_data.index_x[0][0]-1]):lkf_data.index_x[0][-1]+2:lkf_data.red_fac],
                  lkf_data.calc_eps_tot(it),transform=ccrs.PlateCarree(),vmin=0,vmax=0.4)

    lkfs = np.load(lkf_data.lkfpath.joinpath('lkf_%s_%03i.npy' %(lkf_data.netcdf_file.split('/')[-1].split('.')[0],(it+1))),allow_pickle=True)

    for ilkf in lkfs:
        if np.min(ilkf[:,2])<-150 and np.max(ilkf[:,2]>150):
            ilkf[ilkf[:,2]<0,2]+=360
        ax.plot(ilkf[:,2],ilkf[:,3],transform=ccrs.PlateCarree())

    #plt.rcParams.update({'font.size': 14})
    #plt.colorbar(pcm,label='deformation rate/ day$^-1$', location='bottom', shrink=0.5, pad=0.02)
    plt.savefig(f'imgs/{i}.png',dpi=200,bbox_inches='tight')
    plt.close(fig)

In [56]:
import glob
from PIL import Image
def make_gif(frame_folder):
    frames = []
    for i in range(len(lkf_data.indexes)):
        frames.append(Image.open(frame_folder + f'/{i}.png'))
    frame_one = frames[0]

    frame_one.save("lkfs.gif", format="GIF", append_images=frames,
                       save_all=True, duration=250, loop=0)

if __name__ == "__main__":
    make_gif("imgs")